In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from wb import WeightsAndBiases
from sklearn.preprocessing import LabelBinarizer
from random import sample, choice
from fingerprint_vect import GraphFingerprint
from collections import defaultdict
from autograd import grad
from autograd.scipy.misc import logsumexp

import autograd.numpy as np
import networkx as nx
import math

In [3]:
def make_random_graph(nodes, n_edges, features_dict):
    """
    Makes a randomly connected graph. 
    """
    
    G = nx.Graph()
    for n in nodes:
        G.add_node(n, features=features_dict[n])
    
    for i in range(n_edges):
        u, v = sample(G.nodes(), 2)
        G.add_edge(u, v)
        
    return G

In [4]:
# features_dict will look like this:
# {0: array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
#  1: array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
#  2: array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
#  3: array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
#  4: array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
#  5: array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
#  6: array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
#  7: array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
#  8: array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
#  9: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])}

all_nodes = [i for i in range(10)]    
lb = LabelBinarizer()
features_dict = {i:lb.fit_transform(all_nodes)[i] for i in all_nodes}

G = make_random_graph(sample(all_nodes, 6), 5, features_dict)
G.edges(data=True)
# G.nodes(data=True)

[(1, 2, {}), (3, 4, {}), (3, 6, {}), (6, 8, {})]

In [5]:
wb = WeightsAndBiases(n_layers=2, shapes=(10, 20, 10))
# wb

In [6]:
# def score(G):
#     """
#     The regressable score for each graph will be the sum of the 
#     (square root of each node + the sum of its neighbors.)
#     """
#     sum_score = 0
#     for n, d in G.nodes(data=True):
#         sum_score += math.sqrt(n)
        
#         for nbr in G.neighbors(n):
#             sum_score += nbr ** (1/3)
#     return sum_score

def score(G):
    """
    The regressable score for each graph is the number of nodes
    in the graph.
    """
    return len(G.nodes())

score(G)

6

In [7]:
G.nodes(data=True)[0][1]['features'].shape

(10,)

In [8]:
def softmax(X, axis=0):
    """
    The softmax function normalizes everything to between 0 and 1.
    """
    return np.exp(X - logsumexp(X, axis=axis, keepdims=True))

# test softmax:
X = np.random.random((1,10))
softmax(X, axis=1)

array([[ 0.10590253,  0.07747197,  0.06798611,  0.08309748,  0.14523769,
         0.10324535,  0.09085766,  0.11327375,  0.14492305,  0.06800441]])

In [9]:
def relu(X):
    """
    The ReLU - Rectified Linear Unit.
    """
    return X * (X > 0)


# test relu:
X = np.random.normal(0, 1, size=(5, 1))
print(X)
print('')
print(relu(X))

[[ 0.09574597]
 [-0.05016848]
 [ 0.18500304]
 [-1.10497981]
 [ 0.27769682]]

[[ 0.09574597]
 [-0.        ]
 [ 0.18500304]
 [-0.        ]
 [ 0.27769682]]


In [45]:
# Make 1000 random graphs.
syngraphs = []
for i in range(20):
    n_nodes = choice([i for i in range(2, 10)])
    n_edges = choice([i for i in range(1, n_nodes**2)])
    
    G = make_random_graph(sample(all_nodes, n_nodes), n_edges, features_dict)
    syngraphs.append(G)
    
len(syngraphs)

20

In [31]:
# Write a function that computes the feature matrix, and writes the
# indices to the nodes of each graph.
def stacked_node_activations(graphs):
    """
    Note: this function should only be called for computing the
    stacked node activations after initializing the graphs.
    
    Inputs:
    =======
    - graphs: (list) a list of graphs on which to stack their
              feature vectors.
    """
    features = []
    curr_idx = 0
    for g in graphs:
        for n, d in g.nodes(data=True):
            features.append(d['features'])
            g.node[n]['idx'] = curr_idx
            curr_idx += 1
    return np.vstack(features)

# test stacked_node_activations
layers = dict()
layers[0] = stacked_node_activations(syngraphs)
layers[1] = stacked_node_activations(syngraphs)
# layers[1]

In [32]:
# Write a function that gets the indices of each node's neighbors.
def neighbor_indices(G, n):
    """
    Inputs:
    =======
    - G: the graph to which the node belongs to.
    - n: the node inside the graph G.
    
    Returns:
    ========
    - indices: (list) a list of indices, which should (but is not
               guaranteed to) correspond to a row in a large 
               stacked matrix of features.
    """
    indices = []
    for n in G.neighbors(n):
        indices.append(G.node[n]['idx'])
    return indices


# test neighbor_indices
nbr_idxs = neighbor_indices(syngraphs[0], syngraphs[0].nodes()[0])
nbr_idxs

[1, 2, 3, 4, 5, 6]

In [33]:
# Write a function that sums each of the neighbors' activations for a
# given node in a given graph.
def neighbor_activations(G, n, activations_dict, layer):
    """
    Inputs:
    =======
    - G: the graph to which the node belongs to.
    - n: the node inside the graph G
    - activations_dict: a dictionary that stores the node activations 
                        at each layer.
    - layer: the layer at which to compute neighbor activations.
    """
    nbr_indices = neighbor_indices(G, n)
    return np.sum(activations_dict[layer][nbr_indices], axis=0)

neighbor_activations(syngraphs[0], syngraphs[0].nodes()[0], layers, 0)

array([0, 1, 1, 1, 1, 1, 0, 1, 0, 0])

In [34]:
# Write a function that stacks each of the nodes' neighbors
# activations together into a large feature matrix.

def stacked_neighbor_activations(graphs, activations_dict, layer):
    """
    Inputs:
    =======
    - graphs: (list) a list of NetworkX graphs.
    - activations_dict: (dict) a dictionary where keys are the layer
                        number and values are the node activations.
    
    Returns:
    ========
    - a stacked numpy array of neighbor activations
    """
    nbr_activations = []
    for g in graphs:
        for n in g.nodes():
            nbr_acts = neighbor_activations(g, n, activations_dict, layer)
            nbr_activations.append(nbr_acts)
    return np.vstack(nbr_activations)

# stacked_neighbor_activations(syngraphs, layers, 1)

In [46]:
# Write a function that computes the next layers' activations.

def activation(activations_dict, wb, layer, graphs):
    """
    Inputs:
    =======
    - activations_dict: (dict) a dictionary where keys are the layer
                        number and values are the node activations.
    - wb: (wb.WeightsAndBiases) the WB class storing the weights and
          biases.
    - layer: (int) the layer for which to compute the activations.    
    
    Returns:
    ========
    - a stacked numpy array of activations, which can be assigned to
      the activations_dict's next layer if desired (actually it
      should be).
    """
    
    self_acts = activations_dict[layer]
    self_acts = np.dot(self_acts, wb[layer]['self_weights'])

    nbr_acts = stacked_neighbor_activations(graphs, activations_dict, layer)
    nbr_acts = np.dot(nbr_acts, wb[layer]['nbr_weights'])
    
    biases = wb[layer]['biases']
    
    return relu(self_acts + nbr_acts + biases)

print(activation(layers, wb, 0, syngraphs))

KeyError: 'idx'

In [36]:
act = np.dot(stacked_neighbor_activations(syngraphs, layers, 0), wb[0]['nbr_weights']) + wb[0]['biases']
act.shape

(118, 20)

In [37]:
# Write a function that gets the indices of all of the nodes in the
# graph.
def graph_indices(g):
    """
    Returns the row indices of each of the nodes in the graphs.
    """
    return [d['idx'] for _, d in g.nodes(data=True)]

In [38]:
# Write a function that makes the fingerprint used for predictions.
def fingerprint(activations_dict, graphs):
    """
    Computes the final layer fingerprint for each graph.
    
    Inputs:
    =======
    - activations_dict: (dict) a dictionary where keys are the layer
                        number and values are the node activations.
    - graphs: a list of graphs for which to compute the fingerprints.
    
    Returns:
    ========
    - a stacked numpy array of fingerprints, of length len(graphs).
    """
    top_layer = max(activations_dict.keys())
    fingerprints = []
    for g in graphs:
        idxs = graph_indices(g)
        fp = np.sum(activations_dict[top_layer][idxs], axis=0)
        fingerprints.append(fp)
    
    return relu(np.vstack(fingerprints))

# test fingerprint function
fingerprint(layers, syngraphs)

array([[1, 1, 1, 1, 1, 1, 0, 1, 0, 0],
       [1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [1, 1, 1, 1, 0, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 1, 1],
       [0, 0, 1, 1, 1, 0, 1, 1, 1, 1],
       [0, 0, 1, 0, 1, 0, 1, 1, 0, 0],
       [0, 1, 1, 1, 0, 1, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 1, 0, 1, 0, 0],
       [1, 1, 1, 0, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 0, 1, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [0, 1, 0, 0, 0, 1, 0, 1, 0, 1],
       [1, 1, 1, 0, 0, 1, 0, 1, 1, 0],
       [1, 1, 0, 1, 0, 0, 1, 1, 1, 1],
       [0, 1, 1, 1, 0, 1, 1, 1, 1, 1],
       [1, 1, 1, 0, 1, 1, 1, 1, 1, 0]])

In [39]:
# Write a function that makes the forward pass predictions.
def predict(wb_vect, wb_unflattener, activations_dict, graphs):
    """
    Makes predictions.
    
    Change this function for each new learning problem.
    
    Inputs:
    =======
    - wb_vect: (WeightsAndBiases.vect)
    - wb_unfalttener (WeightsAndBiases.unflattener)
    - activations_dict: (dict) a dictionary where keys are the layer
                        number and values are the node activations.
    - graphs: a list of graphs for which to compute the fingerprints.
    
    Returns:
    ========
    - a numpy array of predictions, of length len(graphs).
    """
    
    wb = wb_unflattener(wb_vect)
    for k in sorted(wb.keys()):
        activations_dict[k + 1] = activation(activations_dict, wb, k, graphs)
        # print(activations_dict[k])
    
    top_layer = max(wb.keys())
    
    fps = fingerprint(layers, graphs)
    
    return np.dot(fps, wb[top_layer]['linweights'])

predict(*wb.flattened(), layers, syngraphs)

array([[-0.4140944 ],
       [-0.80994935],
       [-0.05438736],
       [-0.08457893],
       [-0.03971847],
       [-0.35550169],
       [-0.15989237],
       [-0.3462132 ],
       [-0.02066551],
       [-0.02535193],
       [-0.06567089],
       [-0.03222333],
       [-0.22609366],
       [-0.44758711],
       [-0.18338839],
       [-0.04373497],
       [-0.18453728],
       [-0.33261526],
       [-0.21751378],
       [-0.33543044]])

In [40]:
# Write a function that computes the training loss.
def train_loss(wb_vect, wb_unflattener, activations_dict, graphs):
    """
    Computes the training loss as mean squared error.
    
    Inputs:
    =======
    - wb_vect: (WeightsAndBiases.vect)
    - wb_unfalttener (WeightsAndBiases.unflattener)
    - activations_dict: (dict) a dictionary where keys are the layer
                        number and values are the node activations.
    - graphs: a list of graphs for which to compute the fingerprints.

    Returns:
    ========
    - mean squared error.
    """
    
    scores = np.array([score(g) for g in graphs]).reshape((len(graphs), 1))
    # print(scores)
    preds = predict(wb_vect, wb_unflattener, activations_dict, graphs)
    # print(preds)
    return np.sum(np.power(preds - scores, 2)) / len(scores)

train_loss(wb.vect, wb.unflattener, layers, syngraphs)

0.77174070288183994

In [41]:
gradfunc = grad(train_loss, argnum=0)
gradfunc(wb.vect, wb.unflattener, layers, syngraphs)

array([  0.00000000e+00,   4.36545916e-02,   3.01206013e-02, ...,
        -2.99765891e-05,   6.78054181e-02,   0.00000000e+00])

In [42]:
def sgd(grad, wb_vect, wb_unflattener, activations_dict, graphs, callback=None, num_iters=200, step_size=0.1, mass=0.9):
    """
    Stochastic gradient descent with momentum.
    """
    from time import time
    velocity = np.zeros(len(wb_vect))
    for i in range(num_iters):
        start = time()
        print(i)
        g = grad(wb_vect, wb_unflattener, activations_dict, graphs)

        velocity = mass * velocity - (1.0 - mass) * g
        wb_vect += step_size * velocity
        # print(wb_vect)
        print(train_loss(wb_vect, wb_unflattener, activations_dict, graphs))
        end = time()
        print('Epoch Time: {0}'.format(end - start))
    return wb_vect, wb_unflattener

wb_vect, wb_unflattener = sgd(gradfunc, wb.vect, wb.unflattener, layers, syngraphs, num_iters=200, step_size=0.001)

0
0.67276760589
Epoch Time: 0.15809106826782227
1
0.519709997502
Epoch Time: 0.17552399635314941
2
0.365243994256
Epoch Time: 0.14338183403015137
3
0.249816520583
Epoch Time: 0.15077614784240723
4
0.189969966201
Epoch Time: 0.1405642032623291
5
0.179175168473
Epoch Time: 0.2888529300689697
6
0.202457764917
Epoch Time: 0.25779199600219727
7
0.240191970356
Epoch Time: 0.16279006004333496
8
0.275522609796
Epoch Time: 0.17266297340393066
9
0.296764999479
Epoch Time: 0.23338699340820312
10
0.298641869262
Epoch Time: 0.14048004150390625
11
0.281555614684
Epoch Time: 0.14654302597045898
12
0.250107867914
Epoch Time: 0.1505289077758789
13
0.211169974916
Epoch Time: 0.19708895683288574
14
0.171896379056
Epoch Time: 0.2993190288543701
15
0.1382448387
Epoch Time: 0.1716620922088623
16
0.113899536158
Epoch Time: 0.1766970157623291
17
0.100432243778
Epoch Time: 0.12554121017456055
18
0.0965960794042
Epoch Time: 0.16973400115966797
19
0.100463748826
Epoch Time: 0.12432217597961426
20
0.108049144955


In [23]:
train_loss(wb_vect, wb.unflattener, layers, syngraphs)

0.043996281002482249

In [24]:
wb.unflattener(wb.vect)[2]['linweights']

array([[ 0.1676535 ],
       [ 0.03242438],
       [-0.00748493],
       [ 0.66346477],
       [ 0.27239111],
       [-0.09952596],
       [ 0.4714696 ],
       [ 0.07962087],
       [ 0.28633275],
       [-0.04745121]])

In [25]:
scores = [score(g) for g in syngraphs]

In [26]:
preds = predict(wb_vect, wb.unflattener, layers, syngraphs)

In [27]:
[i for i in zip(scores, preds)]

[(2, array([ 2.06262481])),
 (6, array([ 5.92384939])),
 (9, array([ 9.15106495])),
 (5, array([ 5.39515665])),
 (7, array([ 7.002901])),
 (2, array([ 2.00901536])),
 (9, array([ 8.53653948])),
 (8, array([ 7.91750888])),
 (8, array([ 7.99536548])),
 (4, array([ 4.17193318]))]

In [28]:
new_graphs = [make_random_graph(sample(all_nodes, 4), 5, features_dict) for i in range(100)]
# predict(wb_vect, wb.unflattener, layers, new_graphs)
new_graphs[0].nodes(data=True)

stacked_node_activations(new_graphs)

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [29]:
predict(wb_vect, wb.unflattener, layers, new_graphs)

IndexError: index 62 is out of bounds for axis 0 with size 60